In [3]:
## Importing Qiskit libraries
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, AncillaRegister, Aer, execute, assemble
from qiskit_aer import AerSimulator
from qiskit.circuit import Gate
from qiskit.visualization import plot_histogram, circuit_drawer
import matplotlib.pyplot as plt
from qiskit.quantum_info import Operator
from qiskit.circuit.library import MCMT, RYGate, RXGate, PhaseGate
import numpy as np
np.set_printoptions(threshold=np.inf)
import math
from typing import Union
from numpy import linalg
from scipy.linalg import expm
import pickle
from numba import jit

In [2]:
## Settings
d = 1 # of dimensions
M = 2 # of registers (rough estimate for M is the number of particles in the final state, but M should be set so that p_success can be maximized)
n = 1 # of particles in the initial state
N_abs = 1 # of modes for momenta

nqubits = N_abs * d + d + 1
N_s = 2 ** (N_abs + 1) # size of the lattice per one dimension
V = N_s ** d # volume
N = N_abs * d + d + 1 # of qubits per one particle (magnitude of momentum + sign + occupation)
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
bose_marker = 2 ** s - math.factorial(M)

In [3]:
I = np.array([[1, 0], [0, 1]])
H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

In [4]:
def rxyz(theta, sign): #sign = 1, 2, 3 correspond to X, Y, Z
    if sign == 1:
        pauli = X
    elif sign == 2:
        pauli = Y
    elif sign == 3:
        pauli = Z
    return expm(-1j * (theta / 2) * pauli)

In [5]:
#初期状態|0000・・・0>を準備する。
def StateZeros(nqubits):
    State = np.zeros(2**nqubits)
    State[0]=1
    return State

In [6]:
def sendingState2QC(state):
    # 結果を保存
    result_file = 'calculated_result.pkl'
    with open(result_file, 'wb') as f:
        pickle.dump(state, f)

In [51]:
def makeGate(N, nqubit, Gate): # action of Gate to n'th qubit (n = 0...N)
    if nqubit == 0:
        tmp = Gate
    else: tmp = I
    for i in range(N):
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if nqubit == i + 1:
            B_tmp = np.kron(Gate, tmp)
        else: B_tmp = np.kron(I, tmp)
        tmp = B_tmp
    return tmp

# example of use:
# makeGate(N, (N_abs + 1) * d, X)
# makeGate(1, 1, H)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [8]:
def wavePacketPerReg(N_abs):
    sigma = 1
    tmp = 1
    for i in range(N_abs * d):
        theta = math.atan(math.exp(- 2 ** (i - 1) / sigma))
        tmp = np.kron(expm(-1j * theta * Y), tmp)
    tmp = np.kron(H, tmp)
    tmp = np.kron(X, tmp)
    return tmp

In [9]:
def showNonZeroCol(mat):
    list = []
    for i in range(len(mat[0])):
        if mat[i][0] != 0:
            list.append(mat[i][0])
            print(i)
    print(list)

def showNonZeroAll(mat):
    mat_list = []
    raw_list = []
    column_list = []
    for i in range(len(mat[0])):
        for j in range(len(mat[0])):
            if mat[i][j] != 0:
                raw_list.append(i)
                column_list.append(j)
                mat_list.append(mat[i][j])
    print('raw list', raw_list)
    print('column list', column_list)
    print('mat', mat_list)

In [10]:
# def transPosition2Momentum(qmom, sign): # if sign is True, we have to transform qmom to qposition
#     if sign == 'position':
#         qmom = qmom - (N / 2)
#     return qmom

In [11]:
# if dagger is True, make an annihilation operator, if False, an creation operator
# ireg means the number of the register acted
# qmom means that the momentum with sign
def annihilationCreationOpPerReg(qmom, ireg, dagger, N, M):
    if qmom > 0:
        qrow = int(qmom - 1/2 + 2 ** (N-1) + 2 ** N_abs)
    elif qmom < 0:
        qrow = int(np.abs(qmom + 1/2) + 2 ** (N-1))
    tmp_op = np.zeros((2 ** N, 2 ** N))
    if dagger == False:
        tmp_op[0][qrow] = 1
    else:
        tmp_op[qrow][0] = 1
    identity = np.eye(2 ** N)
    if ireg == 0:
        tmp = tmp_op
    else: tmp = identity
    # print(tmp)
    for i in range(M-1):
        # print(i)
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if ireg == i + 1:
            B_tmp = np.kron(tmp_op, tmp)
        else:
            B_tmp = np.kron(identity, tmp)
        tmp = B_tmp
    return tmp

def annihilationCreationOpPerRegInPos(ipos, ireg, dagger, N, M):
    qrow = int(ipos + 2 ** (N-1))
    tmp_op = np.zeros((2 ** N, 2 ** N))
    if dagger == False:
        tmp_op[0][qrow] = 1
    else:
        tmp_op[qrow][0] = 1
    identity = np.eye(2 ** N)
    if ireg == 0:
        tmp = tmp_op
    else: tmp = identity
    # print(tmp)
    for i in range(M-1):
        # print(i)
        B_tmp = np.zeros((2 ** (i + 2), 2 ** (i + 2)))
        if ireg == i + 1:
            B_tmp = np.kron(tmp_op, tmp)
        else:
            B_tmp = np.kron(identity, tmp)
        tmp = B_tmp
    return tmp

def annihilationCreationOp(qmom, dagger, sign):
    tmp_op = np.zeros((2 ** (N * M), 2 ** (N * M)))
    # print(tmp_op.size)
    for ireg in range(M):
        # print(annahilationCreationOpPerReg(qmom, ireg, dagger, N, M).size)
        if sign == 'momentum':
            tmp_op += annihilationCreationOpPerReg(qmom, ireg, dagger, N, M)
        elif sign == 'position':
            tmp_op += annihilationCreationOpPerRegInPos(qmom, ireg, dagger, N, M)
        else: print('sign should be momentum or position!')
    return tmp_op * (1/np.sqrt(M))

# example of use
# N_abs = 2
# N = 4
# M = 1
# annahilationCreationOpPerRegInPos(2, 0, False, N, M)

In [12]:
# annihilationCreationOpPerRegInPos(ipos=3, ireg=0, )

In [13]:
# to ajustment the dimension of Hilbert space with QC algorithnm, we must add ancilla qubits
def addAncilla(mat, s, bose_marker):
    bose_num = s + bose_marker
    anc_eye_mat = np.eye(2 ** int(bose_num))
    return np.kron(anc_eye_mat, mat)

In [14]:
import cmath
# flip alignment code and QFT code
def changeBasis(N):
    tmp = np.eye(2 ** N)
    for i in range(2 ** (N - 2)):
        tmp[2 ** (N - 1) + i][2 ** (N - 1) + i] = 0
        tmp[2 ** (N - 1) + i][2 ** (N - 1) + 2 **(N - 2) - i - 1] = 1
    return tmp

# def dft_matrix(num):
#     A = np.arange(num)
#     B = A.reshape(1, -1)
#     C = A.reshape(-1, 1)
#     M = cmath.e**(-1j * 2 * cmath.pi * B * C  / num)
#     return M * (1/np.sqrt(num))

# def qft(N):
#     return np.kron(np.eye(2), dft_matrix(2 ** (N - 1)))

In [15]:
def dft_matrix(n):
    """
    Computes the matrix representation of the Quantum Fourier Transform (QFT) for n qubits.

    Args:
    - n (int): The number of qubits.

    Returns:
    - np.ndarray: The matrix representation of the QFT.
    """
    # Define the quantum Fourier matrix of size 2^n × 2^n
    qft = np.zeros((2**n, 2**n), dtype=np.complex128)
    for i in range(2**n):
        for j in range(2**n):
            qft[i, j] = np.exp(2j * np.pi * i * j / (2**n)) / np.sqrt(2**n)
    return qft

def qft(N):
    return np.kron(np.eye(2), dft_matrix((N - 1)))

In [16]:
def makeGatePerReg(Gate, M): # make tensor M products of Gate 
    tmp = Gate
    for i in range(M - 1):
        B_tmp = np.zeros((2 ** (N * i), 2 ** (N * i)))
        B_tmp = np.kron(Gate, tmp)
        tmp = B_tmp
    return tmp

In [17]:
def wavePacket(M, n):
    wavePacketGate = wavePacketPerReg(N_abs)
    wavePacketGate2All = makeGatePerReg(wavePacketGate, n)
    for _ in range(M-n):
        for _ in range((N_abs + 1) * d + 1):
            wavePacketGate2All = np.kron(I, wavePacketGate2All)
    return wavePacketGate2All

In [18]:
def tranlateGatePerReg(trans_mat_comp, N_abs, d, M, flag): # flag==1 means position translation and flag==0 momentum translation
    sign = 2 * flag - 1
    nqubit = 2 ** ((N_abs + 1) * d + 1)
    tmp = np.zeros((nqubit, nqubit), dtype='complex128')
    tmp[0][0] = 1
    nhalf = int(nqubit / 2)
    for i in range(nhalf):
        # phi = 2 * np.pi * sign * (((N_abs + 1) * d + 1) - 2 * i) * trans_mat_comp / N_s
        phi = (-2 * np.pi * sign * i * trans_mat_comp) / 2 ** (N_abs + d)
        # print('phi', phi)
        tmp[nhalf + i][nhalf + i] = np.exp(1j * phi)
    return tmp

def transGate(trans_mat, flag, M, n):
    tmp = 1
    for i in range(n):
        tmp = np.kron(tranlateGatePerReg(trans_mat[i][0], N_abs, d, M, flag), tmp)
        # print('tmp', tmp)
    # for unoccupied register
    for _ in range(M-n):
        for _ in range((N_abs + 1) * d + 1):
            tmp = np.kron(I, tmp)
    return tmp

In [19]:
def decimal_numbers_with_bit_set(bit_length, i):
    """
    Generates a list of all decimal numbers where the ith bit is set to 1
    for binary representations of a given bit length.

    Args:
    - bit_length (int): The length of the binary representation.
    - i (int): The index of the bit.

    Returns:
    - list: A list containing all decimal numbers where the ith bit is set to 1.
    """
    decimal_numbers = []
    for num in range(1 << bit_length):  # Iterate through all possible binary representations
        binary_str = bin(num)[2:].zfill(bit_length)  # Convert number to binary string with leading zeros
        if binary_str[bit_length - i - 1] == '1':  # Check if the ith bit is 1
            decimal_numbers.append(int(binary_str, 2))  # Convert binary string to decimal and add to list
    return decimal_numbers

# # Example usage:
# bit_length = 4  # ビット列の長さ
# i = 1           # i番目のビットが1であることを考える

# numbers_list = decimal_numbers_with_bit_set(bit_length, i)
# print("Decimal numbers where the ith bit is set to 1:", numbers_list)

def controlledPhase(M, N_abs, d, delta):
    bit_len = M * ((N_abs + 1) * d + 1)
    N = N_abs * d + d + 1
    tmp = np.eye((2 ** (N * M)), dtype='complex128')
    for ireg in range(M): # for each occpation bit
        bin_occ = (N_abs + 1) * d + ireg * ((N_abs + 1) * d + 1) # binary position of occpation bit
        decimal_occ_list = decimal_numbers_with_bit_set(bit_len, bin_occ)
        for lreg in range(M):
            for jabs in range(N_abs):
                bin_abs = jabs + lreg * ((N_abs + 1) * d + 1) # binary position of abs bit
                # print('bin_abs', bin_abs)
                decimal_abs_list = decimal_numbers_with_bit_set(bit_len, bin_abs)
                for kcomp in range(2 ** (N * M)):
                    # print('decimal_occ_list', decimal_occ_list)
                    # print('decimal_abs_list', decimal_abs_list)
                    # print('kcomp', kcomp)
                    if kcomp in decimal_occ_list and kcomp in decimal_abs_list:
                        # print('i am here')
                        phi = (-2 * np.pi * (jabs + 1) * delta) / (2 ** (N_abs + d) * M)
                        # print('phi', phi)
                        tmp[kcomp][kcomp] *= np.exp(1j * phi)
    return tmp

def freeTimeEvolutionPerReg(delta, N_abs, d):
    # print('vac_list', vac_list)
    nqubit = 2 ** ((N_abs + 1) * d + 1)
    tmp = np.zeros((nqubit, nqubit), dtype='complex128')
    tmp[0][0] = 1
    nquart = int(nqubit / 4)
    nhalf = int(nqubit / 2)
    for i in range(nquart):
        # phi = 2 * np.pi * sign * (((N_abs + 1) * d + 1) - 2 * i) * trans_mat_comp / N_s
        # print(i)
        phi = (-2 * np.pi * (i + 1) * delta) / (2 ** (N_abs + d) * M)
        # print(f'vac_list[%i]', vac_list[nhalf + i])
        # print('phi', phi)
        tmp[nhalf + i][nhalf + i] = np.exp(1j * phi)
        tmp[nhalf + nquart + i][nhalf + nquart + i] = np.exp(1j * phi)
    return tmp

def freeTimeEvolution(delta, N_abs, d, M):
    tmp = 1
    for _ in range(M):
        tmp = np.kron(freeTimeEvolutionPerReg(delta, N_abs, d), tmp)
    return tmp

In [92]:
M = 2
N_abs = 2
d = 1
delta = 1
N = (N_abs + 1) * d + 1
n = 2

nqubits = N_abs * d + d + 1
N_s = 2 ** (N_abs + 1) # size of the lattice per one dimension
V = N_s ** d # volume
N = N_abs * d + d + 1 # of qubits per one particle (magnitude of momentum + sign + occupation)
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
bose_marker = 2 ** s - math.factorial(M)

state = StateZeros((N * M) + (s + bose_marker))
wavePacketGate = wavePacket(M, n)
wavePacketGate = addAncilla(wavePacketGate, s, bose_marker)
state = wavePacketGate @ state
state = addAncilla(freeTimeEvolution(delta, N_abs, d, M), s, bose_marker) @ state
state = addAncilla(controlledPhase(M, N_abs, d, delta), s, bose_marker) @ state
sendingState2QC(state)

In [138]:
def cswap_matrix(bit_length, i, j, k):
    """
    Computes the matrix representation of a CSWAP gate controlled by the k-th bit,
    with i-th and j-th bits being the target bits.

    Args:
    - bit_length (int): The length of the binary representation.
    - i (int): The index of the first target bit.
    - j (int): The index of the second target bit.
    - k (int): The index of the control bit.

    Returns:
    - np.ndarray: The matrix representation of the CSWAP gate.
    """
    # Calculate the size of the matrix
    matrix_size = 2 ** bit_length
    # Initialize the matrix as an identity matrix
    cswap = np.eye(matrix_size)
    i = bit_length - i - 1
    j = bit_length - j - 1
    k = bit_length - k - 1

    # Loop through all possible input combinations
    for n in range(matrix_size):
        # Check if the k-th bit is 1
        if (n >> (bit_length - k - 1)) & 1:
            # If k-th bit is 1, swap the i-th and j-th bits
            # Convert the integer n to binary and pad it with zeros
            binary_str = bin(n)[2:].zfill(bit_length)
            # Swap the i-th and j-th bits
            binary_list = list(binary_str)
            binary_list[i], binary_list[j] = binary_list[j], binary_list[i]
            # binary_list = binary_list[::-1]
            # n = matrix_size - n
            # print('binary_list', binary_list)
            # print('n', n)
            # Recalculate the integer representation
            new_n = int(''.join(binary_list), 2)
            # new_n = matrix_size - new_n
            # Update the corresponding matrix element
            cswap[n, n] = 0
            cswap[new_n, new_n] = 0
            cswap[n, new_n] = 1
            cswap[new_n, n] = 1

    return cswap

# # Example usage:
# bit_length = 3  # 2進数表現のビット数
# i = 2           # i番目のビット
# j = 1           # j番目のビット
# k = 0           # 制御ビットとなるk番目のビット

# # CSWAPゲートの行列を計算
# cswap = cswap_matrix(bit_length, i, j, k)
# print("CSWAP gate matrix:")
# print(cswap)

# import numpy as np

def multi_cnot_matrix(bit_length, control_bits, target_bit):
    """
    Computes the matrix representation of a multi-controlled NOT (CNOT) gate
    with the specified control bits and target bit.

    Args:
    - bit_length (int): The length of the binary representation.
    - control_bits (list of int): The indices of the control bits.
    - target_bit (int): The index of the target bit.

    Returns:
    - np.ndarray: The matrix representation of the multi-CNOT gate.
    """
    # Calculate the size of the matrix
    matrix_size = 2 ** bit_length
    # Initialize the matrix as an identity matrix
    multi_cnot = np.eye(matrix_size)

    # Loop through all possible input combinations
    for n in range(matrix_size):
        # Check if all control bits are 1
        if all((n >> (bit_length - i - 1)) & 1 for i in control_bits):
            # If all control bits are 1, apply NOT operation to the target bit
            # Convert the integer n to binary and pad it with zeros
            binary_str = bin(n)[2:].zfill(bit_length)
            # Apply NOT operation to the target bit
            binary_list = list(binary_str)
            binary_list[target_bit] = str(1 - int(binary_list[target_bit]))
            # Recalculate the integer representation
            new_n = int(''.join(binary_list), 2)
            # Update the corresponding matrix element
            multi_cnot[n, n] = 0
            multi_cnot[new_n, new_n] = 0
            multi_cnot[n, new_n] = 1
            multi_cnot[new_n, n] = 1

    return multi_cnot

# # Example usage:
# bit_length = 3  # 2進数表現のビット数
# control_bits = [0, 1]  # 制御ビットのインデックス
# target_bit = 2  # ターゲットビットのインデックス

# # 複数の制御ビットを持つCNOTゲートの行列を計算
# multi_cnot = multi_cnot_matrix(bit_length, control_bits, target_bit)
# print("Multi-controlled NOT (CNOT) gate matrix:")
# print(multi_cnot)

In [139]:
cswap_matrix(3, 0, 2, 1)

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [154]:
def boseSymmM2n2(N_abs):
    N = (N_abs + 1) * d + 1
    all_bit_len = N * 2 + 1
    particleI = np.eye(2 ** (N * 2), dtype='complex128')
    HGate2anc = np.kron(H, particleI)
    tmp = HGate2anc
    # print('tmp size', tmp.size)
    # swap
    for jbit in range(N):
        # print('CSWAP size', cswap_matrix(all_bit_len, jbit, N + jbit, N * 2 - 1).size)
        # print('swap', jbit, 'and', N + jbit)
        tmp = cswap_matrix(all_bit_len,  jbit, N + jbit, N * 2) @ tmp
    tmp = HGate2anc @ tmp
    return tmp

In [21]:
def boseSymmM3n2(N_abs):
    N = (N_abs + 1) * d + 1
    all_bit_len = N * 3 + 3 + 2
    tmp = makeGate(all_bit_len, N * 3, H) @ makeGate(all_bit_len, N * 3 + 1, H) @ makeGate(all_bit_len, N * 3 + 2, H)
    # swap
    for ianc in range(3):
        for jbit in range(N):
            tmp = cswap_matrix(all_bit_len, N * (ianc % 3) + jbit, N * ((ianc + 1) % 3) + jbit, N * 3 + ianc) @ tmp
    # correct
    control_bits = [N*3, N*3+1, N*3+2]
    # for |1,1,0>
    tmp = makeGate(all_bit_len, N * 3 + 2, X) @ tmp
    tmp = multi_cnot_matrix(all_bit_len, control_bits, N * 3 + 3) @ tmp
    tmp = makeGate(all_bit_len, N * 3 + 2, X) @ tmp
    # for |0,1,0>
    tmp = makeGate(all_bit_len, N * 3 + 2, X) @ tmp
    tmp = makeGate(all_bit_len, N * 3, X) @ tmp
    tmp = multi_cnot_matrix(all_bit_len, control_bits, N * 3 + 4) @ tmp
    tmp = makeGate(all_bit_len, N * 3 + 2, X) @ tmp
    tmp = makeGate(all_bit_len, N * 3, X) @ tmp
    return tmp

In [9]:
def momentum2omega(qmom, m, a_s):
    N_s = 2 ** (N_abs + 1) # number of lattice per dimension
    pmom = ((2 * np.pi) / (N_s * a_s)) * qmom
    omega = np.sqrt(pmom ** 2 + m ** 2)
    return omega

In [23]:
def sqij(qmom, ireg, jreg, m, a_s):
    omega = momentum2omega(qmom, m, a_s)
    zq = (1/2) * np.log(omega)
    A = annihilationCreationOpPerReg(qmom, ireg, True, N, M)
    B = annihilationCreationOpPerReg(-qmom, jreg, True, N, M)
    C = annihilationCreationOpPerReg(-qmom, jreg, False, N, M)
    D = annihilationCreationOpPerReg(qmom, ireg, False, N, M)
    ln_sq = -((2 * zq)/M) * (A @ B - C @ D) # 2 means that q and -q are calulated simultaneously
    sq_ij = expm(ln_sq)
    return sq_ij

# squeezing operation for verifying QC algorithms
def sqDebug(m, a_s):
    sq = np.eye(2 ** (N * M), dtype='complex128')
    for qmom in range(2 ** (N_abs * d)-1, -1, -1):
        for ireg in range(M-1, -1, -1):
            for jreg in range(M-1, -1, -1):
                if ireg != jreg:
                    # print('qmom, ireg, jreg', qmom, ireg, jreg)
                    sq @= sqij(qmom + 1/2, ireg, jreg, m, a_s)
    return sq

def squeezeOpPerMom(qmom, m, a_s):
    omega = momentum2omega(qmom, m, a_s)
    zq = (1/2) * np.log(omega)
    A = annihilationCreationOp(qmom, True, 'momentum')
    # print('A is', A)
    B = annihilationCreationOp(-qmom, True, 'momentum')
    # print('B is', B)
    C = annihilationCreationOp(-qmom, False, 'momentum')
    # print('C is', C)
    D = annihilationCreationOp(qmom, False, 'momentum')
    # print(linalg.norm(A @ B, 2))
    ln_sq = -2 * zq * (A @ B - C @ D)
    sq = expm(ln_sq)
    return sq

def squeezeOpQProd(m, a_s): # corresponds to \prod exp{\sum_{i} a^{i} + a^{i}^{dagger}}
    sq = np.eye(2 ** (N * M))
    for q in range(2 ** (N_abs * d)-1, -1, -1):
        # print(q)
        # print(squeezeOpPerMom(q+1/2))
        # print(squeezeOpPerMom(-(q+1/2)))
        # print('qmom', q + 1/2)
        sq @= squeezeOpPerMom(q + 1/2, m, a_s)
        # sq @= squeezeOpPerMom(-(q + 1/2))
    return sq

def squeezingOp(m, a_s):
    ln_sq = np.zeros((2 ** (N * M), 2 ** (N * M)))
    for s in range(2):
        sign = (-1) ** s
        for q in range(2 ** (N_abs * d)-1, -1, -1):
            qmom = (q + 1/2) * sign
            # print('qmom', qmom)
            omega = momentum2omega(qmom, m, a_s)
            zq = (1/2) * np.log(omega)
            A = annihilationCreationOp(qmom, True, 'momentum')
            B = annihilationCreationOp(-qmom, True, 'momentum')
            C = annihilationCreationOp(-qmom, False, 'momentum')
            D = annihilationCreationOp(qmom, False, 'momentum')
            ln_sq += -zq * (A @ B - C @ D)
    return expm(ln_sq)

In [15]:
N_abs = 2
qmom = (6/2)
# print('qmom', qmom)
omega = momentum2omega(qmom, m=0, a_s=1)
zq = (1/2) * np.log(omega)
zq/2

0.21426195334940482

In [24]:
# # pos_trans_mat = np.zeros((n, d))
# # pos_trans_mat[0][0] = 1
# state = StateZeros(N * M + (s + bose_marker))
# # print('state size', len(state))
# wavePacketGate = wavePacket(M, n)
# wavePacketGate = addAncilla(wavePacketGate, s, bose_marker)
# state = wavePacketGate @ state

# # translateGate = transGate(pos_trans_mat, True, M, n)
# # translateGate = addAncilla(translateGate, s, bose_marker)
# # print('translateGate size', translateGate.size)
# # state = translateGate @ state

# delta = 1
# freeEvolutionGate = freeTimeEvolution(delta, N_abs, d, M)
# freeEvolutionGate = addAncilla(freeEvolutionGate, s, bose_marker)
# state = freeEvolutionGate @ state
# print('state', state)
# sendingState2QC(state)

In [25]:
# ## Scheme of Squeezing operation
# M = 2
# N_abs = 1
# d = 1
# N = N_abs * d + d + 1
# n = 1
# s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
# tau_0 = 0.5 # Wigner delay
# tau_I = 0 # length of time of interaction
# delta = 1 # time step
# m_0 = 0 # chosen to represent a relevant enrgy scale in the weak coupling regime
# tau = 0
# m_t = m_0
# lambda_t = 1
# a_s = 1

# m_ren = m_0
# lambda_ren = 1
# free_num = int((N_abs + math.log2(n)) * d)
# bose_num = int(s + 1)
# state = StateZeros(((N * M) + (free_num + bose_num)))
# count = 0
# mat = addAncilla(sqij(qmom=1/2, ireg=0, jreg=1, m=0, a_s=1), N_abs, n, d, s)

# m = 0
# a_s = 1
# # mat = addAncilla(sqDebug(m, a_s), N_abs, n, d, s)
# # state = np.dot(mat, state)

# list = []
# for i in range(len(mat[0])):
#     if mat[i][0] != 0:
#         list.append(mat[i][0])
#         print(i)
# list

In [26]:
def intUnij(ireg, jreg, kreg, lreg, ipos, Delta):
    ln_exp = np.eye(2 ** (N * M), dtype='complex128')
    A = annihilationCreationOpPerRegInPos(ipos, ireg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, ireg, True, N, M)
    B = annihilationCreationOpPerRegInPos(ipos, jreg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, jreg, True, N, M)
    C = annihilationCreationOpPerRegInPos(ipos, kreg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, kreg, True, N, M)
    D = annihilationCreationOpPerRegInPos(ipos, lreg, False, N, M) + annihilationCreationOpPerRegInPos(ipos, lreg, True, N, M)
    ln_exp = -1j * Delta * (A @ B @ C @ D)
    # print('I am here')
    # showNonZeroAll(ln_exp)
    return expm(ln_exp)

# interaction Hamiltonian for verifying QC algorithms
def intUDebug(delta, lam):
    Delta = (delta * lam) / (96 * (M ** 2))
    int_u = np.eye(2 ** (N * M), dtype='complex128')
    for ipos in range(2 ** (N_abs * d + d)-1, -1, -1):
        for ireg in range(M):
            for jreg in range(M):
                for kreg in range(M):
                    for lreg in range(M):
                        int_u @= intUnij(ireg, jreg, kreg, lreg, ipos, Delta)
    return int_u

def phi(n):
    A = annihilationCreationOp(n, True, 'position')
    B = annihilationCreationOp(n, False, 'position')
    phiGate = (1/np.sqrt(2)) * (A + B)
    return phiGate

# exact value of interaction Hamiltonian time evolution
def intU(delta, lam):
    ln_UI = 0
    for ipos in range(2 ** (N_abs * d + d)):
        ln_UI += -1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos))
        # print(expm(-1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos))))
    uI = expm(ln_UI)
    return uI

# approximation for commutation error in [a_m, a_n^dagger]
def intUNProd(delta, lam):
    uI = np.eye(2 ** (N * M), dtype='complex128')
    for ipos in range(2 ** (N_abs * d + d)):
        # print(expm(-1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos))))
        uI @= expm(-1j * ((delta * lam) / 24) * (phi(ipos) @ phi(ipos) @ phi(ipos) @ phi(ipos)))
    return uI

In [27]:
# The same pattern with QC algorithms
def intUDebug2(delta, lam):
    Delta = (delta * lam) / (96 * (M ** 2))
    int_u = np.eye(2 ** (N * M), dtype='complex128')
    for ipos in range(2 ** (N_abs * d + d) -1, -1, -1):
        # print('ipos', ipos)
    #     dummy = 0
    # if dummy == 0:
    #     ipos = 0
        # all differ
        for ireg in range(M-1, -1, -1):
            for jreg in range(M-1, -1, -1):
                for kreg in range(M-1, -1, -1):
                    for lreg in range(M-1, -1, -1):
                        if ireg < jreg and jreg < kreg and kreg < lreg:
                            # print('all differ;', 'ireg', ireg, 'jreg', jreg, 'kreg', kreg, 'lreg', lreg)
                            int_u @= intUnij(ireg, jreg, kreg, lreg, ipos, 24 * Delta)

        # 2 indices match
        for ireg in range(M-1, -1, -1):
            for jreg in range(M-1, -1, -1):
                for kreg in range(M-1, -1, -1):
                    if ireg != jreg and ireg != kreg and jreg < kreg:
                        # print('2 indices;', 'ireg', ireg, 'jreg', jreg, 'kreg', kreg)
                        int_u @= intUnij(ireg, ireg, jreg, kreg, ipos, 12 * Delta)

        # 2 pairs of indices match
        for ireg in range(M-1, -1, -1):
            for jreg in range(M-1, -1, -1):
                if ireg < jreg:
                    # print('2 pairs match;', 'ireg', ireg, 'jreg', jreg)
                    int_u @= intUnij(ireg, ireg, jreg, jreg, ipos , 6 * Delta)

        # 3 indices match
        for ireg in range(M-1, -1, -1):
            for jreg in range(M-1, -1, -1):
                if ireg < jreg:
                    # print('3 indices;', 'ireg', ireg, 'jreg', jreg)
                    int_u @= intUnij(ireg, ireg, ireg, jreg, ipos, 8 * Delta)

        # 4 indices match
        for ireg in range(M-1, -1, -1):
            # print('4 indices match;', 'ireg', ireg)
            int_u @= intUnij(ireg, ireg, ireg, ireg, ipos, Delta)
            
    return int_u

In [28]:
def commutation(phi1, phi2):
    return phi1 @ phi2 -phi2 @ phi1

In [30]:
# This code shows each phi(i.e. a_n and a_n'^dagger)deos not commute
# M = 5
# count = 0
# comm = commutation(phi(1), phi(2))
# for i in range(phi(1)[0].size):
#     for j in range(phi(1)[0].size):
#         if comm[i][j] != 0:
#             print(comm[i][j])
#             # print(comm[i][j])
#             count += 1
#         if count == 1:
#             break
#     if count == 1:
#         break
# print(count)

## A. 初期状態の準備

In [145]:
def initStatePre(N_abs, d, n, M, pos_trans_mat, mom_trans_mat):
    N = (N_abs + 1) * d + 1
    state = StateZeros(N * M + (s + bose_marker))
    print('state size', len(state))
    
    # localization of wave packet
    wavePacketGate = wavePacket(M, n)
    wavePacketGate = addAncilla(wavePacketGate, s, bose_marker)
    state = wavePacketGate @ state

    # Change basis
    C = addAncilla(makeGatePerReg(changeBasis(N), M), s, bose_marker)
    state = np.dot(C, state)
    # print('i am here')

    # translation in position space
    translateGate = transGate(pos_trans_mat, True, M, n)
    # print('i am here')
    # print('transGate', translateGate)
    translateGate = addAncilla(translateGate, s, bose_marker)
    # print('translateGate size', translateGate.size)
    state = translateGate @ state

    # QFT
    Q = addAncilla(makeGatePerReg(qft(N), M), s, bose_marker)
    state = np.dot(Q, state)

    # translation in momentum space
    translateGate = transGate(mom_trans_mat, False, M, n)
    translateGate = addAncilla(translateGate, s, bose_marker)
    # print('translateGate size', translateGate.size)
    state = translateGate @ state

    # inverse QFT
    Q = addAncilla(makeGatePerReg(qft(N), M), s, bose_marker)
    invQ = np.linalg.inv(Q)
    state = np.dot(invQ, state)

    # Change basis
    C = addAncilla(makeGatePerReg(changeBasis(N), M), s, bose_marker)
    state = np.dot(C, state)

    # bose symmetrization
    if M ==  2 and n ==  2:
        B = boseSymmM2n2(N_abs)
        state = B @ state

    return state

In [146]:
M = 2
N_abs = 1
d = 1
n = 2

s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
bose_marker = 2 ** s - math.factorial(M)//math.factorial(M - n)

pos_trans_mat = np.zeros((n, d))
pos_trans_mat[0][0] = 1

mom_trans_mat = np.zeros((n, d))
mom_trans_mat[0][0] = 1

state = initStatePre(N_abs, d, n, M, pos_trans_mat, mom_trans_mat)
# print(state)
sendingState2QC(state)

state size 128


In [90]:
cswap_matrix(3, 0, 1, 2)

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

## B. 時間発展

In [30]:
def freePart(state, delta, N_abs, d, n, M):
    state = addAncilla(freeTimeEvolution(delta, N_abs, d, M), s, bose_marker) @ state
    state = addAncilla(controlledPhase(M, N_abs, d, delta), s, bose_marker) @ state
    return state

In [31]:
def intPart(state, lambda_t, m_t, a_s, N_abs, d, n, M):
    N = (N_abs + 1) * d + 1
    # inverse sq
    invS0 = addAncilla(squeezingOp(m_t, a_s), s, bose_marker)
    invS = np.linalg.inv(invS0)
    state = np.dot(invS, state)

    # Change basis
    C = addAncilla(makeGatePerReg(changeBasis(N), M), s, bose_marker)
    state = np.dot(C, state)

    # QFT
    Q = addAncilla(makeGatePerReg(qft(N), M), s, bose_marker)
    state = np.dot(Q, state)

    ## Interaction Hamiltonian
    U_I = addAncilla(intU(delta, lambda_t), s, bose_marker)
    state = np.dot(U_I, state)

    # inverse QFT
    Q = addAncilla(makeGatePerReg(qft(N), M), s, bose_marker)
    invQ = np.linalg.inv(Q)
    state = np.dot(invQ, state)

    # Change basis
    C = addAncilla(makeGatePerReg(changeBasis(N), M), s, bose_marker)
    state = np.dot(C, state)

    # Squeezing operation
    S = addAncilla(squeezingOp(m_t, a_s), s, bose_marker)
    state = np.dot(S, state)
    return state

In [55]:
M = 3
N_abs = 1
d = 1
N = N_abs * d + d + 1
n = 1
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
bose_marker = int(2 ** s - (math.factorial(M)/math.factorial(M - n)))
tau_0 = 1 # Wigner delay
tau_I = 0 # length of time of interaction
delta = 2 # time step
m_0 = 0 # chosen to represent a relevant energy scale in the weak coupling regime
tau = 0
m_t = m_0
lambda_t = 1
a_s = 1

pos_trans_mat = np.zeros((n, d))
pos_trans_mat[0][0] = 1

mom_trans_mat = np.zeros((n, d))
mom_trans_mat[0][0] = 1

m_ren = m_0
lambda_ren = 1
count = 0

state = StateZeros(N * M + (s + bose_marker))

while tau <= tau_0 + tau_I:
    state = freePart(state, delta, N_abs, d, n, M)
    state = intPart(state, lambda_t, m_t, a_s, N_abs, d, n, M)

    count += 1
    print('We count', count, 'steps')

    tau += delta
    if tau < tau_0: # turn on interaction adiabatically
        lambda_t = lambda_ren * (tau / tau_0)
        m_t = m_0 * (1 - (tau / tau_0)) + m_ren * (tau / tau_0)
    elif tau_0 <= tau : # interaction completely
        m_t = m_ren
        lambda_t = lambda_ren

# print(state)

sendingState2QC(state)

We count 1 steps


In [32]:
# M = 1
# N_abs = 1
# d = 1
# N = N_abs * d + d + 1
# n = 1
# s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
# bose_marker = 2 ** s - math.factorial(M)
# tau_0 = 0.5 # Wigner delay
# tau_I = 0 # length of time of interaction
# delta = 1 # time step
# m_0 = 0 # chosen to represent a relevant energy scale in the weak coupling regime
# tau = 0
# m_t = m_0
# lambda_t = 1
# a_s = 1

# pos_trans_mat = np.zeros((n, d))
# pos_trans_mat[0][0] = 1

# mom_trans_mat = np.zeros((n, d))
# mom_trans_mat[0][0] = 0

# m_ren = m_0
# lambda_ren = 1
# count = 0

# while tau <= tau_0 + tau_I:
#     # print('count', count)
#     # inverse Squeezing operation
#     # invS0 = addAncilla(squeezeOpQProd(m_t, a_s), N_abs, n, d, s)
#     invS0 = addAncilla(squeezingOp(m_t, a_s), s, bose_marker)
#     # invS0 = addAncilla(sqDebug(m_t, a_s), N_abs, n, d, s)
#     invS = np.linalg.inv(invS0)
#     state = np.dot(invS, state)

#     # Change basis
#     C = addAncilla(makeGatePerReg(changeBasis(N), M), s, bose_marker)
#     state = np.dot(C, state)

#     # QFT
#     Q = addAncilla(makeGatePerReg(qft(N), M), s, bose_marker)
#     state = np.dot(Q, state)

#     ## Interaction Hamiltonian
#     # # print(lambda_t)
#     U_I = addAncilla(intU(delta, lambda_t), s, bose_marker)
#     # U_I = addAncilla(intUNProd(delta, lambda_t), N_abs, n, d, s)
#     # U_I = addAncilla(intUDebug(delta, 100 * lambda_t), N_abs, n, d, s)
#     # U_I = addAncilla(intUDebug2(delta, 100 * lambda_t), N_abs, n, d, s)
#     state = np.dot(U_I, state)

#     # inverse QFT
#     Q = addAncilla(makeGatePerReg(qft(N), M), s, bose_marker)
#     invQ = np.linalg.inv(Q)
#     state = np.dot(invQ, state)

#     # Change basis
#     C = addAncilla(makeGatePerReg(changeBasis(N), M), s, bose_marker)
#     state = np.dot(C, state)

#     # Squeezing operation
#     # S = addAncilla(squeezeOpQProd(m_t, a_s), N_abs, n, d, s)
#     S = addAncilla(squeezingOp(m_t, a_s), s, bose_marker)
#     # S = addAncilla(sqDebug(m_t, a_s), N_abs, n, d, s)
#     state = np.dot(S, state)

#     count += 1
#     print('We count', count, 'steps')

#     tau += delta
#     if tau < tau_0: # turn on interaction adiabatically
#         lambda_t = lambda_ren * (tau / tau_0)
#         m_t = m_0 * (1 - (tau / tau_0)) + m_ren * (tau / tau_0)
#     elif tau_0 <= tau : # interaction completely
#         m_t = m_ren
#         lambda_t = lambda_ren

# # print(state)

# sendingState2QC(state)

## all parts

In [156]:
M = 2
N_abs = 2
d = 1
N = N_abs * d + d + 1
n = 2
s = math.ceil(math.log2(math.factorial(M)/math.factorial(M - n))) # of ancilla qubits for Bose symm.
bose_marker = int(2 ** s - (math.factorial(M)/math.factorial(M - n)))
tau_0 = 1 # Wigner delay
tau_I = 0 # length of time of interaction
delta = 1.5 # time step
m_0 = 0 # chosen to represent a relevant energy scale in the weak coupling regime
tau = 0
m_t = m_0
lambda_t = 1
a_s = 1

pos_trans_mat = np.zeros((n, d))
pos_trans_mat[0][0] = 3

mom_trans_mat = np.zeros((n, d))
mom_trans_mat[0][0] = 1

m_ren = m_0
lambda_ren = 1
count = 0

state = initStatePre(N_abs, d, n, M, pos_trans_mat, mom_trans_mat)

while tau <= tau_0 + tau_I:
    state = freePart(state, delta, N_abs, d, n, M)
    state = intPart(state, lambda_t, m_t, a_s, N_abs, d, n, M)

    count += 1
    print('We count', count, 'steps')

    tau += delta
    if tau < tau_0: # turn on interaction adiabatically
        lambda_t = lambda_ren * (tau / tau_0)
        m_t = m_0 * (1 - (tau / tau_0)) + m_ren * (tau / tau_0)
    elif tau_0 <= tau : # interaction completely
        m_t = m_ren
        lambda_t = lambda_ren

# print(state)

sendingState2QC(state)

state size 512
We count 1 steps


In [70]:
list = []
for i in range(len(state)):
    if state[i] != 0:
        list.append(state[i])
        print(i)
list

0
36
45
54
63
260
288
325
360
390
432
455
504


[(-0.043786295847682566+0.05881607295685135j),
 (0.3784679668764735-0.31304283968701807j),
 (0.04492923509584299+0.2541934177653032j),
 (-0.12784607756012956-0.045393187832698884j),
 (0.05210956737109052-0.050171237852961056j),
 (0.37849241302396036-0.3130628523860462j),
 (0.3784796452293447-0.31303394873462803j),
 (0.04493204234064055+0.2542097829083181j),
 (0.04492214197906194+0.2541964499233758j),
 (-0.12785428624445308-0.04539614928106796j),
 (-0.12784562381881137-0.045397216576785616j),
 (0.05210956737109051-0.050171237852961056j),
 (0.05210956737109051-0.050171237852961056j)]